In [1]:
from zumi.zumi import Zumi
from zumi.util.screen import Screen
import serial
import csv
import time
import os


In [2]:
zumi = Zumi()
screen = Screen()

Starting Zumi 
Pi Zero I2C is available
Verified Pi Zero is the same
Gyroscope previously calibrated
Zumi board detected
Compass detected
OLED Screen detected
Gyroscope & Accelerometer detected
Zumi battery level is low  2 %
Please charge me!


In [ ]:
zumi.control_motors(10,13)

In [ ]:
zumi.stop()

Approach with very little movements and a lot of praying involved

In [ ]:
start_time = time.time()
zumi.reset_gyro()
turns = 0
while True:
    # [123, 255, 1, 255, 100,20]
    ir_readings = zumi.get_all_IR_data()
    bottom_right = ir_readings[1]
    bottom_left = ir_readings[3]
    front_right = ir_readings[0]
    front_left = ir_readings[5]

    # Threshold for detecting black (190-200) vs grey (100-120)
    threshold = 150  # Adjust as needed

    if bottom_right < threshold and bottom_left < threshold:
        zumi.reverse(speed = 20, duration = 0.25)
        zumi.turn_right(45)
        ir_readings = zumi.get_all_IR_data()
        bottom_right = ir_readings[1]
        bottom_left = ir_readings[3]
        if bottom_right < threshold or bottom_left < threshold:
            zumi.signal_left_on()
            zumi.turn_left(90)
            time.sleep(0.3)
            zumi.forward(speed = 20, duration = 0.1)
            zumi.signal_left_off()
        else:
            zumi.signal_right_on()
            zumi.forward(speed = 20, duration = 0.1)
            zumi.signal_right_off()

    elif bottom_right > threshold and bottom_left > threshold:
        zumi.control_motors(10,13)
        if zumi.read_z_angle() > 40 or zumi.read_z_angle() < -40:
            turns += 1
        zumi.reset_gyro()

    elif bottom_right < threshold:
        turn_start = time.time()
        curve_start = True
        # Both sensors detect black → Go straight
        zumi.control_motors(20,0)
    
    elif bottom_left < threshold:
        # Left sensor on black, right on grey → Slight left turn
        zumi.control_motors(0,20)  # Left wheel slower
    if front_right < 15 and front_left < 15:
        zumi.stop()
        break
        
# Record end time
end_time = time.time()

csv_filename = "/home/pi/Dashboard/user/RobotChallenge/My_Projects/Jupyter/zumi_runtime.csv"  # Passe den Pfad nach Bedarf an

# Open and write to the CSV file
with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([start_time, end_time, turns])
    writer.writerow("")

In [ ]:
for i in range(0,10):
    ir_readings = zumi.get_all_IR_data()
    bottom_right = ir_readings[1]
    bottom_left = ir_readings[3]
    
    message = "    IR readings        "
    message = message + str(bottom_right) + ", " + str(bottom_left)
    screen.draw_text(message)
    time.sleep(0.1)
screen.draw_text_center("Done!")

In [62]:
zumi.reset_gyro()
for i in range(0,50):
    current_angle = int(zumi.read_z_angle())
    message = " Z-Angle reading           "
    message = message + str(current_angle)
    screen.draw_text(message)
    time.sleep(0.05)
    
print("Done")
screen.draw_text_center("Done")

Done
